In [ ]:
import numpy as np
import csv
from itertools import product, combinations
import tensorflow.compat.v1 as tf 
tf.compat.v1.disable_eager_execution()
import sys
sys.path.append("../")
import copy
import pandas as pd
from tensorflow.python.platform import flags
import time
from adf_data.census import census_data
from adf_data.credit import credit_data
from adf_data.compas import compas_data
from adf_data.default import default_data
from adf_data.bank import bank_data
from adf_data.heart import heart_data
from adf_model.tutorial_models import dnn
from adf_utils.utils_tf import model_prediction, model_argmax , layer_out
from adf_utils.config import census, credit, bank, compas, default, heart

FLAGS = flags.FLAGS
       
def m_instance( sample, sens_params, conf):
    index = []
    m_sample = []
    for sens in sens_params:
        index.append([i for i in range(conf.input_bounds[sens-1][0], conf.input_bounds[sens-1][1]+1)])
      
    for ind in list(product(*index)):     
        temp = sample.copy()
        for i in range(len(sens_params)):
            temp[0][sens_params[i]-1] = ind[i]
        m_sample.append(temp)
    return np.array(m_sample)
    
def clustering(probs,m_sample, sens_params):
    epsillon=0.025
    cluster_dic = {}
    cluster_dic['Seed']=m_sample[0][0]

        #  to avoid k = Max + 1 
    for i in range(len(probs)):

    
        if probs[i] == 1.0:
            if (int( probs[i] / epsillon ) -1) not in cluster_dic.keys():
             
                cluster_dic[ (int( probs[i] / epsillon ) -1)] = [ [m_sample[i][0][j-1] for j in sens_params]]
           
            else:
                cluster_dic[ (int( probs[i] / epsillon ) -1)].append( [m_sample[i][0][j-1] for j in sens_params] )

                       
        elif int( probs[i] / epsillon ) not in cluster_dic.keys():
                cluster_dic[ int( probs[i] / epsillon )] = [ [m_sample[i][0][j-1] for j in sens_params] ]
           
        else:
                cluster_dic[ int( probs[i] / epsillon)].append( [m_sample[i][0][j-1] for j in sens_params] )

    return cluster_dic  

    
def pred_prob(sess, x, preds, m_sample, input_shape):
        #global probs
        probs = model_prediction(sess, x, preds, np.array(m_sample).reshape(len(m_sample),
                                                                            input_shape[1]))[:,1:2].reshape(len(m_sample))
        return probs        
        
def neuron_locator(sess, model, samples, layer_number,model_path, input_shape, nb_classes, dataset, sens_params, update_list,X,Y ):
        if  sess._closed:
            config = tf.ConfigProto()
            config.gpu_options.per_process_gpu_memory_fraction = 0.8
            sess = tf.Session(config = config)
            x = tf.placeholder(tf.float32, shape = input_shape)
            y = tf.placeholder(tf.float32, shape = (None, nb_classes))
            model = dnn(input_shape, nb_classes)   

            preds = model(x)
            saver = tf.train.Saver()
            #model_path = model_path + dataset + "/test.model"
            saver.restore(sess, model_path)
        num_layers=len(model.layers)
        #print(update_list[0])
        feed_dic = {}
        for neuron in range(len(update_list)):
            time1 = time.time()              
            for layer in range(0,num_layers - 1,2):
                if layer == 0:
                    l = model.layers[layer].fprop(samples.astype('float32'))
                else:
                    l = model.layers[layer].fprop(r)                   
                if layer + 1 == (layer_number * 2) - 1:
                    indices = []
                    for instance in range(l.shape[0]):                       
                        indices.append([ instance, 0, neuron])       
                    updates = [ update_list[ neuron ] ] * l.shape[0]
                    #print('update',update_list[ neuron ])
                    r = model.layers[layer + 1].fprop(l , indices, updates)
                    #print(sess.run(r)[0])
                else:
                    r = model.layers[layer + 1].fprop(l)
            feed_dic[neuron] = r
        all_probs = sess.run(feed_dic)
        out_dic={}
        for key in all_probs.keys():
#             print(key)
            probs = np.array(all_probs[key]).reshape((90,2))[:,1:].reshape((90))
            clus = clustering(probs,samples, sens_params)
            out_dic[key]= [len(clus) - 1 ]
#             print(neuron)
#             input(model_acc(sess, model,key,X,Y,layer_number,num_layers,update_list ))
            
            #out_dic[key].append(model_acc(sess, model,key,X,Y,layer_number,num_layers,update_list ))
        sess.close()
        tf.reset_default_graph()
        return out_dic

def get_rate(lay_name, layer_output):
        def get_distance(vec1, vec2, size):
            return abs(vec1-vec2).sum() / size
            
#         config = tf.ConfigProto()
#         config.gpu_options.per_process_gpu_memory_fraction = 0.8
#         sess = tf.Session(config = config)
#         x = tf.placeholder(tf.float32, shape = input_shape)
#         y = tf.placeholder(tf.float32, shape = (None, nb_classes))
#         model = dnn(input_shape, nb_classes)   

#         preds = model(x)
#         saver = tf.train.Saver()
#         model_path = model_path + dataset + "/test.model"
#         saver.restore(sess, model_path)
        max_dis = 0
        epsillon = 10 ** -7
        num_samples = len(layer_output[lay_name])

        layer_ind = np.where(np.array(list(layer_output.keys())) == lay_name)[0][0]
        for ind in range(layer_ind):
            temp_dis = 0
            if 'ReLU' in np.array(list(layer_output.keys()))[ind]:
                layer_name = np.array(list(layer_output.keys()))[ind]
                layer_size  = len(layer_output[layer_name][0][0])
                distances = np.zeros((num_samples,num_samples))
                for i in combinations(range(num_samples),2):
                    distances[i[0],i[1]] = get_distance(layer_output[layer_name][i[0]],layer_output[layer_name][i[1]],layer_size)
                if distances.max()> max_dis:
                    max_dis = distances.max()
        distances = np.zeros((num_samples,num_samples))
        for i in combinations(range(num_samples),2):
            distances[i[0],i[1]] = get_distance(layer_output[lay_name][i[0]],layer_output[lay_name][i[1]],len(layer_output[lay_name][0][0]))
        cur_dis = distances.max()

        change_rate = (cur_dis - max_dis ) / (max_dis + epsillon)      
        return change_rate 

    
    
def model_acc(sess, model,model_path,input_shape, nb_classes,
              dataset, sens_params,neuron,X,Y,layer_number,num_layers,update_list):
        if  sess._closed:
                config = tf.ConfigProto()
                config.gpu_options.per_process_gpu_memory_fraction = 0.8
                sess = tf.Session(config = config)
                x = tf.placeholder(tf.float32, shape = input_shape)
                y = tf.placeholder(tf.float32, shape = (None, nb_classes))
                model = dnn(input_shape, nb_classes)   

                preds = model(x)
                saver = tf.train.Saver()
                #model_path = model_path + dataset + "/test.model"
                saver.restore(sess, model_path)
        feed_dic = {}
         
        for layer in range(0,num_layers - 1,2):
            if layer == 0:
                l = model.layers[layer].fprop(X.astype('float32'))
            else:
                l = model.layers[layer].fprop(r)
           
            if layer + 1 == (layer_number * 2) - 1:

                indices = []
                for instance in range(l.shape[0]):                       
                    indices.append([ instance, neuron])
                
                updates = [ update_list[ neuron ] ] * l.shape[0]
                
                r = model.layers[layer + 1].fprop(l , indices, updates)
                #input(sess.run(r)[0])
            else:
                r = model.layers[layer + 1].fprop(l)             

        all_probs = sess.run(r)
        out_class = []
        for out in all_probs:
            out_class.append(np.argmax(out))
        truth_val = []
        for tr in Y:
                truth_val.append(np.argmax(tr))
        acc=0
        for i in range(len(out_class)):
            if out_class[i]==truth_val[i]:
                acc +=1
        accuracy = round(acc/len(out_class),3)
        sess.close()
        tf.reset_default_graph()
        #print(accuracy)
        return accuracy    
#-------------------------------------------
    
def dnn_fair_testing(dataset, sens_params, model_path):

    data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas":compas_data, 
            "default": default_data, "heart":heart_data}
    data_config = {"census":census, "credit":credit, "bank":bank, "compas":compas, "default":default,
                  "heart":heart}
    # prepare the testing data and model
    X, Y, input_shape, nb_classes = data[dataset]()
    tf.set_random_seed(1234)

    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.8
#     config = tf.ConfigProto(device_count = {'GPU': 0})
#     config.allow_soft_placement= True


    sess = tf.Session(config=config)
    x = tf.placeholder(tf.float32, shape=input_shape)
    y = tf.placeholder(tf.float32, shape=(None, nb_classes))
    model = dnn(input_shape, nb_classes)   

    preds = model(x)
    saver = tf.train.Saver()
    model_path = model_path + dataset + "/test.model"
    saver.restore(sess, model_path)
    input_df=pd.read_csv('../results/census/final_disc_instances_k_maxk_40.csv',header='infer')
    sample_df=input_df.copy()
    sample_df_rand = sample_df.sample(n=6300,axis=0,random_state=np.random.RandomState())
    sample_df_maxk = sample_df.sort_values(by='#k',ascending=False).head(700)
    sample_df = pd.concat([sample_df_rand,sample_df_maxk])
    ini_k_samples = sample_df['#k']
    sample_df=sample_df.drop(columns=['#disc','#k']) 
    samples = sample_df.to_numpy()
    print(len(samples))
    np.save('../results/census/samples_6300_700.npy', samples)
    update_df=pd.read_csv('../results/census/dataset_layer2_out.csv')
#     update_list_mean = update_df.mean()
#     update_list_max = update_df.max()
#     update_list_mode = update_df.mode()
    #-----------------------------
    update_min = update_df.min()
    update_max = update_df.max()
    update_mean = update_df.mean()
    update_std = update_df.std()
    update_list=[]
    update_list.append(update_min)
    rand_point = np.sort(np.random.random(4))
    for i in range(4):
        update_list.append(rand_point[i] * update_mean)
    update_list.append(update_mean)
    rand_point = np.sort(np.random.random(3))
    for i in range(3):
        update_list.append(update_mean + rand_point[i] * update_std)
    update_list.append(update_mean + update_std +  np.random.random(1)[0] * update_std)        
    update_list.append(update_max)
    #----------------------------------
    layer_number = 2 #supposed to come from layer locator
    layer_size = model.layers[(layer_number*2) - 1].input_shape[1]
    layer_name = model.layers[(layer_number*2) - 1]
    num_layers = len(model.layers)

    all_dic={}
    accu_neuron={}
    acc_try={}
    sample_ind = 0
    for sample in samples:       
        update_list_man = np.array([0] * layer_size)
        m_samples=m_instance( np.array([sample]), sens_params, data_config[dataset])
        change_dic={}
        for i in range(11):
            update_list_man = update_list[i]               
            x=neuron_locator(sess, model, m_samples, layer_number,model_path,
                           input_shape, nb_classes, dataset, sens_params, update_list_man,X,Y )
            if sample_ind ==0:
                accu_neuron={}
                for neuron in range(len(update_list_man)):
                    accu_neuron[neuron] = model_acc(sess, model,model_path,
                                     input_shape, nb_classes, dataset, sens_params,
                                     neuron,X,Y,layer_number,num_layers,update_list_man)
                acc_try[i]= accu_neuron                 
            change_dic[i]=x   
        all_dic[sample_ind]=change_dic
        print(sample_ind)
        sample_ind+=1
        
     
    #print(model_acc(sess, model,neuron,X,Y,layer_number,num_layers,update_list_man ))
    np.save('../results/census/ini_k_samples_6300_700.npy', np.array(ini_k_samples))
    np.save('../results/census/accu_dic_6300_700.npy', acc_try) 
    np.save('../results/census/all_dic_6300_700.npy', all_dic)   
              
 
        
def main(argv=None):
    dnn_fair_testing(dataset = FLAGS.dataset, 
                     sens_params = FLAGS.sens_params,
                     model_path  = FLAGS.model_path)


if __name__ == '__main__':
    flags.DEFINE_string("dataset", "census", "the name of dataset")
    flags.DEFINE_string('model_path', '../models/', 'the path for testing model')
    flags.DEFINE_integer('cluster_num', 4, 'the number of clusters to form as well as the number of centroids to generate')
    flags.DEFINE_list('sens_params',[9,8,1],'sensitive parameters index.1 for age, 9 for gender, 8 for race')
    tf.app.run()


In [3]:
import numpy as np
import pandas as pd
accu_dic=dict(np.load('../results/census/accu_dic_900_100.npy',allow_pickle=True).item())  
all_dic=dict(np.load('../results/census/all_dic_900_100.npy',allow_pickle=True).item())
ini_k_random_samples = np.load('../results/census/ini_k_samples_900_100.npy')
df_acc =  pd.DataFrame(columns=['force','neuron','acc'],dtype='float')
df_analysis = pd.DataFrame(columns=['sample','force','neuron','K'],dtype='int32')
for sample in all_dic.keys():
    for force in all_dic[0].keys():
        for neuron in all_dic[0][0].keys():
            df_analysis = df_analysis.append({'sample':sample,'force':force,
                                              'neuron':neuron,'K':all_dic[sample][force][neuron][0]},
                                               ignore_index=True)
            if sample == 0:
                df_acc = df_acc.append({'force':force,'neuron':neuron,
                                'acc':accu_dic[force][neuron]},
                                           ignore_index=True)
                
df_test =  df_analysis.copy() 
df_test['acc'] = list(df_acc['acc']) * len(df_test['sample'].unique())
df_test['init_k'] = 0
len_each_try = len(df_test.loc[df_test['sample']==0])
for sample in range(0,len(df_test['sample']),len_each_try):
    for i in range(len_each_try):
        df_test['init_k'][sample+i]=ini_k_random_samples[int(sample/len_each_try)]
R_act   = []
R_deact = []
diff_R  = []
ini_acc = 0.881
acc_e = 0.05
for neuron in range(32):
    k_deact = df_test.loc[(df_test['neuron']==neuron) & (df_test['force']==0) & \
                          (df_test['acc'] >= ini_acc - acc_e)]['K'].mean()
    k_act   = df_test.loc[(df_test['neuron']==neuron) & (df_test['force'] > 0) & \
                          (df_test['acc'] >= ini_acc - acc_e)]['K'].mean()
    k_init  = df_test.loc[(df_test['neuron']==neuron) & (df_test['acc'] >= ini_acc - acc_e)]['init_k'].mean()
    R_act_temp   = (k_act - k_init) / k_init
    R_deact_temp = (k_deact - k_init) / k_init
    diff_R_temp  = R_act_temp - R_deact_temp
    R_act.append(R_act_temp)
    R_deact.append(R_deact_temp)
    diff_R.append(diff_R_temp)

for i in range(len(diff_R)):
    print(i,' ',round(diff_R[i],3))
            

<ipython-input-3-1444e681885d>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['init_k'][sample+i]=ini_k_random_samples[int(sample/len_each_try)]


0   -0.003
1   0.0
2   0.0
3   -0.054
4   -0.006
5   -0.007
6   0.012
7   -0.047
8   0.0
9   -0.181
10   -0.023
11   0.0
12   -0.034
13   -0.062
14   0.075
15   0.146
16   0.0
17   -0.023
18   -0.065
19   -0.029
20   0.0
21   -0.043
22   -0.051
23   -0.023
24   0.124
25   -0.083
26   -0.072
27   -0.001
28   -0.02
29   -0.0
30   -0.02
31   -0.046


In [7]:
R_act   = []
R_deact = []
diff_R  = []
ini_acc = 0.881
acc_e = 0.05
for neuron in range(32):
    k_deact = df_test.loc[(df_test['neuron']==neuron) & (df_test['force']<=1) & \
                          (df_test['acc'] >= ini_acc - acc_e)]['K'].mean()
    k_act   = df_test.loc[(df_test['neuron']==neuron) & (df_test['force'] > 1) & \
                          (df_test['acc'] >= ini_acc - acc_e)]['K'].mean()
    k_init  = df_test.loc[(df_test['neuron']==neuron) & (df_test['acc'] >= ini_acc - acc_e)]['init_k'].mean()
    R_act_temp   = (k_act - k_init) / k_init
    R_deact_temp = (k_deact - k_init) / k_init
    diff_R_temp  = R_act_temp - R_deact_temp
    R_act.append(R_act_temp)
    R_deact.append(R_deact_temp)
    diff_R.append(diff_R_temp)

for i in range(len(diff_R)):
    print(i,' ',round(diff_R[i],3))
    

0   -0.003
1   0.0
2   0.0
3   -0.057
4   -0.006
5   -0.008
6   -0.018
7   -0.048
8   0.0
9   -0.193
10   -0.021
11   0.005
12   -0.034
13   -0.069
14   0.082
15   0.153
16   0.0
17   -0.022
18   -0.07
19   -0.027
20   0.0
21   -0.041
22   -0.052
23   -0.023
24   0.133
25   -0.085
26   -0.078
27   0.002
28   -0.02
29   -0.0
30   -0.022
31   -0.046


In [27]:
for i in range(len(diff_R)):
    print(i,' ',round(diff_R[i],3))

0   -0.003
1   0.0
2   0.0
3   -0.054
4   -0.006
5   -0.007
6   0.012
7   -0.047
8   0.0
9   -0.181
10   -0.023
11   0.0
12   -0.034
13   -0.062
14   0.075
15   0.146
16   0.0
17   -0.023
18   -0.065
19   -0.029
20   0.0
21   -0.043
22   -0.051
23   -0.023
24   0.124
25   -0.083
26   -0.072
27   -0.001
28   -0.02
29   -0.0
30   -0.02
31   -0.046


In [6]:
for i in range(32):
    print('Neuron ',i)
    print('Num samples force 0 = ',len(df_test.loc[(df_test['force']<=1) & \
                      (df_test['neuron']==i) & (df_test['acc']>0.83)]['K']))
    print(df_test.loc[(df_test['force']==0) & \
                      (df_test['neuron']==i) & (df_test['acc']>0.83)]['K'].mean(),'     ',df_test.loc[(df_test['force']==0) &\
                (df_test['neuron']==i) & (df_test['acc']>0.82)]['acc'].mean())
#     print(df_test.loc[(df_test['force']==0) &\
#                 (df_test['neuron']==i) & (df_test['acc']>0.82)]['acc'].mean())
    
    print('Numb samples force 1 = ',len(df_test.loc[(df_test['force']>1) & \
                      (df_test['neuron']==i) & (df_test['acc']>0.83)]['K']))
    print(df_test.loc[(df_test['force']>=1) & \
                      (df_test['neuron']==i) & (df_test['acc']>0.83)]['K'].mean(),'     ',df_test.loc[(df_test['force']>=1) &\
                    (df_test['neuron']==i) & (df_test['acc']>0.83)]['acc'].mean())
#     print(df_test.loc[(df_test['force']>=1) &\
#                     (df_test['neuron']==i) & (df_test['acc']>0.82)]['acc'].mean())
    print('------------')

Neuron  0
Num samples force 0 =  2000
19.9       0.878
Numb samples force 1 =  8000
19.849       0.8781111111111112
------------
Neuron  1
Num samples force 0 =  2000
20.075       0.8810000000000001
Numb samples force 1 =  9000
20.075       0.8810000000000002
------------
Neuron  2
Num samples force 0 =  2000
20.075       0.8810000000000001
Numb samples force 1 =  9000
20.075       0.8810000000000002
------------
Neuron  3
Num samples force 0 =  2000
18.268       0.8729999999999998
Numb samples force 1 =  8000
17.18111111111111       0.8531111111111113
------------
Neuron  4
Num samples force 0 =  2000
20.077       0.8810000000000001
Numb samples force 1 =  8000
19.962666666666667       0.8810000000000001
------------
Neuron  5
Num samples force 0 =  2000
19.735       0.8810000000000001
Numb samples force 1 =  8000
19.598333333333333       0.880888888888889
------------
Neuron  6
Num samples force 0 =  2000
17.701       0.84
Numb samples force 1 =  9000
17.9354       0.8577
-----------

In [73]:
# df_analysis_pivot=pd.pivot_table(df_analysis, values=['K','Accuracy'], index=['sample', 'force','neuron'])
# df_analysis_pivot.loc[( 0, 0, 1),['K','Accuracy']]

# accu_dic=dict(np.load('../results/census/accu_dic.npy',allow_pickle=True).item())
# all_dic=dict(np.load('../results/census/all_dic.npy',allow_pickle=True).item())
k_dic={}
acc_dic={}
for neuron in range(32):

    N_k=pd.DataFrame(columns=range(0,10),dtype='int32')
    N_acc=pd.DataFrame(columns=range(0,10),dtype='int32')
    n_k=df_test[df_test['neuron']==neuron]
    for force in range(10):

        N_k[force]=n_k[n_k['force']==force]['K'].reset_index()['K']
        N_acc[force]=n_k[n_k['force']==force]['acc'].reset_index()['acc']
    
    k_dic[neuron]=N_k
    acc_dic[neuron]=N_acc

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# for key in k_dic.keys():
#     k_dic[key]=pd.DataFrame(scaler.fit_transform(k_dic[key]))
#     acc_dic[key]=pd.DataFrame(scaler.fit_transform(acc_dic[key]))   

# df_analysis['Accuracy']=scaler.fit_transform(pd.DataFrame(df_analysis['Accuracy']))
# df_analysis['K']=scaler.fit_transform(pd.DataFrame(df_analysis['K']))

In [ ]:
x=df_test.loc[df_test['sample']==2]
x['K']=scaler.fit_transform(pd.DataFrame(x['K']))
x['acc']=scaler.fit_transform(pd.DataFrame(x['acc']))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import matplotlib.pyplot as plt
x=df_test.loc[df_test['sample']==13]
x['K']=scaler.fit_transform(pd.DataFrame(x['K']))
x['acc']=scaler.fit_transform(pd.DataFrame(x['acc']))
for n in range(10):
    plt.plot(x[ df_test['force']==n]['neuron'], 
             scaler.fit_transform(pd.DataFrame(x[ x['force']==n]['acc'])),label = "acc")
    plt.plot(x[ x['force']==n]['neuron'], 
             scaler.fit_transform(pd.DataFrame(x[ x['force']==n]['K'])),label = "K")
    plt.xlabel('Neuron')
    plt.ylabel('K')
    plt.legend()
    plt.show()

In [ ]:
import plotly.express as px
x1=x.loc[x['force']==0]['K']
x2 =x.loc[x['force']==0]['acc']
fig = px.line( x=range(32), y=x1, title="Unsorted Input") 
fig.show()